In [1]:
import os
os.chdir("../")

import pytesseract
import easyocr
from PIL import Image

In [2]:
filename = 'data/menues/menu.jpg'
filename_text = 'data/menues_text/002.txt'

In [3]:
image = Image.open(filename)
with open(filename_text, 'r') as f:
    text = f.read().replace('\n', ' ').replace(".", "").replace(",", "").replace(":", "").replace(";", "").replace("!", "").replace("?", "").replace("…", "").replace("|", "").replace("/", "").replace("-", "")
text

'La cuisine de Linnéa & Xsénia Délicieuse Cuisine Internationale  PetitDéjeuner  Œufs et toast $300 Weisswurst et bretzel $500  Déjeuner  Sandwich $600 Cheeseburger ou hamburger $700 Poisson & champignons $800  Dessert  Gâteau au chocolat $400 Salade de fruits $200  Boissons  Thé $100 Limonade $200 Lait $300'

In [4]:
reader = easyocr.Reader(['fr'])
result_ocr = reader.readtext(filename, detail=0) 
print("EasyOCR: \n", "\n".join(result_ocr).replace('\n', ' '))#.replace(".", "").replace(",", "").replace(":", "").replace(";", "").replace("!", "").replace("?", "").replace("…", "").replace("|", "").replace("/", "").replace("-", ""))

EasyOCR: 
 Tout 4tMEnu Nos Sandwichs avec boisson 33cl Servis en_pain normal ou tortilla avec frites et sauces qu choix CHICKEN Crudités, poulet curry, cheddar, œuf 6,90 € VEGETARIEN Crudités, galette de pomme de terre, cheddar, poivron, oignon 6,90 € AMERICAIN Crudités 2 steaks, cheddar, œuf 7,50 € TRIPLEX Crudités, 3 steaks; bacon, 3 cheddars 7,90 € CROQ CHEF Crudités, poulet curry, crème fraiche emmental, fromage, cheddar 7,90 € ROYAL Crudités 2 steaks, cheddar , galette de pomme de terre, œuf 7,90 € Nos Burgers Garnis avec salade_tomates fraîches et cornichons Servis_avec frites et sauces au choix CHEESE BURGER 1 steak, 1 cheddar 6,50 € DOUBLE_CHEESE 2 steaks, 2 cheddars 7,50 € FISH BURGER Poisson pané, 1 cheddar 6,90 € BIG BURGER 2 steaks, cheddars, galette 7,90 € de pomme de terre Nos Croqs BACON BURGER 1 steak, 1 cheddar, bacon, œuf 6,90 € Garnis de salade et tomates CHICKEN Poulet pané, 1 cheddar; œuf 6,90 € Servis avec frites et sauces au choix BURGER CROQ1 2 pains de mie, jam

In [5]:
result_tes = pytesseract.image_to_string(image, lang='fra')
print("Tesseract: \n", result_tes)#.replace('\n', ' ').replace(".", "").replace(",", "").replace(":", "").replace(";", "").replace("!", "").replace("?", "").replace("…", "").replace("|", "").replace("/", "").replace("-", ""))

Tesseract: 
 os Sandwich

Servis en pain normal ou tortilla avec frites et
sauces au choix

CHICKEN Crudités, poulet curry, cheddar, œuf 6,90€
VEGETARIEN Crudités, galette de pomme de terre, cheddar, poivron, oignon 6,90€
AMERICAIN Crudités , 2 steaks, cheddar, œuf 7,50€
TRIPLE X Crudités, 3 steaks, bacon, 3 cheddars 7,90€
CROQ CHEF Crudités, poulet curry, crème fraiche , emmental, fromage, cheddar 7,90€
ROYAL Crudités , 2 steaks, cheddar, galette de pomme de terre, œuf 7,90€

Nos Burgers

Garnis avec salade, tomates fraîches et cornichons
Servis avec frites et sauces au choix

2 pains de mie, 1 steak, 6,90€
cheddar, lardons

CHEESE BURGER 1 steak, 1 cheddar 6,50€
DOUBLE CHEESE 2 steaks, 2 cheddars 7,50€
FISH BURGER Poisson pané, 1 cheddar 6,90€
BIG BURGER 2 steaks, 2 cheddars, 1 galette 7,90€ N C

de pomme de terre OS roqs
BACON BURGER 1 steak, 1 cheddar, bacon, œuf 6,90€ Garnis de salade et tomates
CHICKEN Poulet pané, 1 cheddar, œuf 6,90€  Servis avec frites et sauces au choix
BURGE

In [6]:
# Load model directly
from transformers import AutoProcessor, AutoModelForImageTextToText

local_model_path = "./models/Qwen2-VL-7B-Instruct"
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct", cache_dir=local_model_path)
model = AutoModelForImageTextToText.from_pretrained("Qwen/Qwen2-VL-7B-Instruct", cache_dir=local_model_path)

/Users/guillaume/Documents/Hackathon/SmartMenue/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46
Loading checkpoint shards: 100%|██████████| 5/5 [00:27<00:00,  5.50s/it]


In [7]:
from transformers import AutoProcessor, AutoModelForImageTextToText
local_model_path = "./models/Qwen2-VL-7B-Instruct"
processor = AutoProcessor.from_pretrained(local_model_path)
model = AutoModelForImageTextToText.from_pretrained(local_model_path)
inputs = processor(image, text, return_tensors="pt", padding="max_length", max_length=128, truncation=True)
outputs = model(**inputs)
predicted_ids = outputs.logits.argmax(-1)
result = processor.decode(predicted_ids[0], skip_special_tokens=True)
print("Qwen: \n", result)#.replace('\n', ' ').replace(".", "").replace(",", "").replace(":", "").replace(";", "").replace("!", "").replace("?", "").replace("…", "").replace("|", "").replace("/", "").replace("-", "")


ValueError: Unrecognized model in ./models/Qwen2-VL-7B-Instruct. Should have a `model_type` key in its config.json, or contain one of the following strings in its name: albert, align, altclip, audio-spectrogram-transformer, autoformer, bark, bart, beit, bert, bert-generation, big_bird, bigbird_pegasus, biogpt, bit, blenderbot, blenderbot-small, blip, blip-2, bloom, bridgetower, bros, camembert, canine, chameleon, chinese_clip, chinese_clip_vision_model, clap, clip, clip_text_model, clip_vision_model, clipseg, clvp, code_llama, codegen, cohere, conditional_detr, convbert, convnext, convnextv2, cpmant, ctrl, cvt, dac, data2vec-audio, data2vec-text, data2vec-vision, dbrx, deberta, deberta-v2, decision_transformer, deformable_detr, deit, depth_anything, deta, detr, dinat, dinov2, distilbert, donut-swin, dpr, dpt, efficientformer, efficientnet, electra, encodec, encoder-decoder, ernie, ernie_m, esm, falcon, falcon_mamba, fastspeech2_conformer, flaubert, flava, fnet, focalnet, fsmt, funnel, 